<a href="https://colab.research.google.com/github/kmt19/siim_otani/blob/master/drive2gcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
"""

     |████████████████████████████████| 993kB 2.9MB/s 


In [0]:
"""
#https://drive.google.com/drive/u/2/my-drive/
dir_id = "1zHVf-O0xXgd47GaVjkpFRUuq59-4Ci5h"

# フォルダ内にあるファイル名とファイルIDを表示する
file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % dir_id})
for f in file_list :
  new_list=drive.ListFile({'q': "'%s' in parents and trashed=false" % f[0]["id"]}).GetList()
  a=drive.CreateFile({'id': new_list[0]["id"],'q': "'%s' in parents and trashed=false" % new_list[0]["id"]})
  print(a)
  a.GetContentFile('sample.dcm')
  break
!ls
"""

GoogleDriveFile({'id': '1KAnyCwq8q8shjI_3wO2fN2P5s07kbssh', 'q': "'1KAnyCwq8q8shjI_3wO2fN2P5s07kbssh' in parents and trashed=false"})


FileNotDownloadableError: ignored

In [1]:
from google import colab
colab.drive.mount('/content/gdrive')
!ls 'gdrive/My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
'Colab Notebooks'    dicom-images-train   手法のアイデア.gdoc
 dicom-images-test   train.csv


In [2]:
!pip install pydicom
!git clone https://github.com/kmt19/siim_otani/
!ls

     |████████████████████████████████| 7.0MB 3.4MB/s 
Cloning into 'siim_otani'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 32 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.
gdrive	sample_data  siim_otani


In [3]:
import os
import pydicom
from glob import glob
from tqdm import tqdm
import pandas as pd
from siim_otani.mask_functions import mask2rle,rle2mask
from siim_otani.pipeline import SIIMDataset
DRIVE_DIR = 'gdrive/My Drive/'
folders = os.listdir(DRIVE_DIR)
print(folders)


['手法のアイデア.gdoc', 'dicom-images-test', 'dicom-images-train', 'Colab Notebooks', 'train.csv']


In [4]:
train_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-train')
test_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-test')
print(os.listdir(train_dicom_dir)[0])
a_dir = os.path.join(train_dicom_dir, os.listdir(train_dicom_dir)[0])
print(os.listdir(a_dir))
b_dir = os.path.join(a_dir, os.listdir(a_dir)[0])
print(os.listdir(b_dir))
"""
ここから各Directory内のlistを読み込み、
BASE_DIR = 'train_dicom_dir/%s/%s/' % (file_list, file_list2)を
for文でもらってきて最終的にpathを保存したdataframeを作成する。
"""
for filename in os.listdir(train_dicom_dir):
  BASE_DIR = 'train_dicom_dir/%s/%s/' % (filename, filename)


1.2.276.0.7230010.3.1.2.8323329.13318.1517875244.629616
['1.2.276.0.7230010.3.1.3.8323329.13318.1517875244.629615']
['1.2.276.0.7230010.3.1.4.8323329.13318.1517875244.629617.dcm']


In [5]:
train_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-train')
test_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-test')
#print(len(os.listdir(train_dicom_dir)))
#print(len(os.listdir(test_dicom_dir)))
train_glob = f'{train_dicom_dir}/*/*/*.dcm'
test_glob = f'{test_dicom_dir}/*/*/*.dcm'

exclude_list = []
train_names = [f for f in tqdm(glob(train_glob))]
test_names = [f for f in tqdm(glob(test_glob))]


100%|██████████| 1377/1377 [00:00<00:00, 1246612.69it/s]


In [23]:
data_files={}
phase = ["train","test"]
data_files["train"] = pd.DataFrame(columns=['Path', 'ID'])
data_files["test"] = pd.DataFrame(columns=['Path'])
SEGMENTATION = DRIVE_DIR + 'train.csv'
anns = pd.read_csv(SEGMENTATION)
train_list=os.listdir(train_dicom_dir)

for i, filename in tqdm(enumerate(train_list)):
  train_glob = f'{train_dicom_dir}/{filename}/*/*.dcm'
  data_files["train"].loc[i] = [glob(train_glob), anns.loc[i][1]]

test_list=os.listdir(test_dicom_dir)  
for i, filename in tqdm(enumerate(test_list)):
  test_glob = f'{test_dicom_dir}/{filename}/*/*.dcm'
  data_files["test"].loc[i] = [glob(test_glob)]
#traindataloader=SIIMDataset(train_names,SEGMENTATION)
"""
for folder in folders:
  new_folder = path + '/' + folder
  new_folder = new_folder + '/' + os.listdir(new_folder)[0]
  new_file = new_folder + '/' + os.listdir(new_folder)[0]
  ds = pydicom.dcmread(new_file)
  break
  """

10712it [00:50, 211.87it/s]
1377it [00:03, 354.68it/s]


"\nfor folder in folders:\n  new_folder = path + '/' + folder\n  new_folder = new_folder + '/' + os.listdir(new_folder)[0]\n  new_file = new_folder + '/' + os.listdir(new_folder)[0]\n  ds = pydicom.dcmread(new_file)\n  break\n  "

In [10]:
train_glob = f'{train_dicom_dir}/{os.listdir(train_dicom_dir)[0]}/*/*.dcm'
print(glob(train_glob))

['gdrive/My Drive/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.13318.1517875244.629616/1.2.276.0.7230010.3.1.3.8323329.13318.1517875244.629615/1.2.276.0.7230010.3.1.4.8323329.13318.1517875244.629617.dcm']
